# Generate KD Cache (K=64 or K=128)

This notebook generates higher-fidelity KD caches for 2-bit QAT training.
Richer teacher signal helps with quality recovery.

**Available configurations:**
- **K=64**: Good balance of quality vs size (~6 GB)
- **K=128**: Maximum fidelity for fine-tuning (~12 GB)

**Changes from default (K=32):**
- `--top_k 64/128` (up from 32)
- `--random_negatives 512/1024` (up from 256)
- Optionally: longer context `--max_length 256`

In [ ]:
# ============================================================
# SETUP: Clone repo and install dependencies
# ============================================================

!git clone https://github.com/anemll/qwen3_apple_style_2bit_qat_lora.git
%cd qwen3_apple_style_2bit_qat_lora
!pip install -q transformers accelerate datasets sentencepiece

In [ ]:
# ============================================================
# CONFIG - Choose K=64 or K=128
# ============================================================

MODEL_NAME = 'Qwen/Qwen3-0.6B'

# ============================================================
# TOP-K CONFIGURATION (uncomment ONE option)
# ============================================================

# Option A: K=64 - Good balance of quality vs size (~6 GB)
TOP_K = 64
RANDOM_NEGATIVES = 512

# Option B: K=128 - Maximum fidelity for fine-tuning (~12 GB)
# TOP_K = 128
# RANDOM_NEGATIVES = 1024  # Also increase negatives for K=128

# ============================================================
# Other parameters
# ============================================================
MAX_LENGTH = 128        # Sequence length (can increase to 256 for longer context)
NUM_SEQUENCES = 20000   # Number of sequences to cache

# Output cache name (auto-generated from params)
CACHE_NAME = f"alpaca_chat_think_both_L{MAX_LENGTH}_K{TOP_K}_R{RANDOM_NEGATIVES}"
CACHE_DIR = f"caches/{CACHE_NAME}"

print(f"Will generate cache: {CACHE_NAME}")
print(f"  - Top-K: {TOP_K}")
print(f"  - Random negatives: {RANDOM_NEGATIVES}")
print(f"  - Max length: {MAX_LENGTH}")
print(f"  - Sequences: {NUM_SEQUENCES}")
print(f"  - Estimated size: ~{TOP_K * 0.1:.0f} GB")

In [ ]:
# ============================================================
# MOUNT GOOGLE DRIVE (for saving cache)
# ============================================================

from google.colab import drive
drive.mount('/content/drive')

# Create destination directory
!mkdir -p /content/drive/MyDrive/qwen3_caches

In [ ]:
# ============================================================
# GENERATE KD CACHE (K=64 or K=128 based on config)
# ============================================================
# K=64:  ~30-60 minutes depending on GPU
# K=128: ~60-90 minutes depending on GPU

%cd /content/qwen3_apple_style_2bit_qat_lora

!python scripts/precompute_teacher_topk.py \
  --teacher_model_name_or_path {MODEL_NAME} \
  --dataset_name tatsu-lab/alpaca \
  --dataset_split train \
  --dataset_format alpaca_chat \
  --enable_thinking both \
  --max_length {MAX_LENGTH} \
  --topk {TOP_K} \
  --rand_neg {RANDOM_NEGATIVES} \
  --num_sequences {NUM_SEQUENCES} \
  --output_dir {CACHE_DIR} \
  --batch_size 32 \
  --device auto

In [ ]:
# ============================================================
# VERIFY CACHE
# ============================================================

import os
import json

if os.path.isdir(CACHE_DIR):
    # Count shards
    shards = [f for f in os.listdir(CACHE_DIR) if f.startswith('shard_')]
    print(f"[cache] Generated {len(shards)} shards")
    
    # Check meta.json
    meta_path = os.path.join(CACHE_DIR, 'meta.json')
    if os.path.exists(meta_path):
        with open(meta_path) as f:
            meta = json.load(f)
        print(f"[cache] Meta info:")
        print(f"  - top_k: {meta.get('top_k')}")
        print(f"  - random_negatives: {meta.get('random_negatives')}")
        print(f"  - max_length: {meta.get('max_length')}")
        print(f"  - total_sequences: {meta.get('total_sequences')}")
    
    # Calculate size
    total_size = sum(os.path.getsize(os.path.join(CACHE_DIR, f)) for f in os.listdir(CACHE_DIR))
    print(f"[cache] Total size: {total_size / (1024**3):.2f} GB")
else:
    print(f"[cache] ERROR: {CACHE_DIR} not found")

In [ ]:
# ============================================================
# COPY CACHE TO GOOGLE DRIVE (folder, no compression)
# ============================================================

import os

DEST_DIR_GD = "/content/drive/MyDrive/qwen3_caches"

if os.path.isdir(CACHE_DIR):
    print(f"[save] Copying {CACHE_NAME} to Google Drive...")
    !rsync -ah --info=progress2 {CACHE_DIR}/ {DEST_DIR_GD}/{CACHE_NAME}/
    
    # Verify
    gd_path = f"{DEST_DIR_GD}/{CACHE_NAME}"
    if os.path.isdir(gd_path):
        num_files = len(os.listdir(gd_path))
        print(f"[save] Successfully saved to Google Drive: {num_files} files")
    else:
        print(f"[save] ERROR: Failed to copy to Google Drive")
else:
    print(f"[save] ERROR: {CACHE_DIR} not found. Run cache generation first.")

In [ ]:
# ============================================================
# OPTIONAL: Also create compressed archive
# ============================================================

import os

CREATE_ARCHIVE = False  # Set to True if you want .tgz as well

if CREATE_ARCHIVE and os.path.isdir(CACHE_DIR):
    print(f"[archive] Compressing {CACHE_NAME}...")
    !tar -zcvf {CACHE_NAME}.tgz -C caches {CACHE_NAME}
    
    # Copy archive to Google Drive
    print(f"[save] Copying {CACHE_NAME}.tgz to Google Drive...")
    !rsync -ah --info=progress2 {CACHE_NAME}.tgz {DEST_DIR_GD}/
    
    # Verify
    archive_size = os.path.getsize(f"{DEST_DIR_GD}/{CACHE_NAME}.tgz")
    print(f"[save] Archive saved: {archive_size / (1024**3):.2f} GB")

## Summary

**Cache configurations:**

| Config | Top-K | Negatives | Est. Size | Use Case |
|--------|-------|-----------|-----------|----------|
| K=64   | 64    | 512       | ~6 GB     | Good balance for 2-bit QAT |
| K=128  | 128   | 1024      | ~12 GB    | Maximum fidelity for fine-tuning |

**To use this cache in training notebooks:**

For K=64:
```python
CACHE_DIR_CHAT = 'caches/alpaca_chat_think_both_L128_K64_R512'
```

For K=128:
```python
CACHE_DIR_CHAT = 'caches/alpaca_chat_think_both_L128_K128_R1024'
```

**Load from Google Drive:**
```python
CACHE_NAME = "alpaca_chat_think_both_L128_K64_R512"  # or K128_R1024
!rsync -ah --info=progress2 /content/drive/MyDrive/qwen3_caches/{CACHE_NAME}/ caches/{CACHE_NAME}/
```